In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0

2026-02-14 22:11:03.396528: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 22:11:03.575218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-14 22:11:04.672956: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import os
print(os.getcwd())

/run/media/shaunakmishra25/DATA/CB/AI ML/BRISC/Attention Unet


In [3]:
print(os.getcwd())

IMG_SIZE = 256
BATCH_SIZE = 2
EPOCHS = 25

BASE_PATH = "../archive/brisc2025/segmentation_task"

TRAIN_IMG = f"{BASE_PATH}/train/images"
TRAIN_MASK = f"{BASE_PATH}/train/masks"

TEST_IMG = f"{BASE_PATH}/test/images"
TEST_MASK = f"{BASE_PATH}/test/masks"

PRED_BASE = f"{BASE_PATH}/predictions/attention_unet/test"

PRED_MASK_DIR = f"{PRED_BASE}/masks"
OVERLAY_DIR = f"{PRED_BASE}/overlays"

os.makedirs(PRED_MASK_DIR, exist_ok=True)
os.makedirs(OVERLAY_DIR, exist_ok=True)


/run/media/shaunakmishra25/DATA/CB/AI ML/BRISC/Attention Unet


In [4]:
def load_image_mask(img_path, mask_path):

    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = img_to_array(img).astype("float32") / 255.0

    mask = load_img(
        mask_path,
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode="grayscale"
    )
    mask = img_to_array(mask).astype("float32") / 255.0
    mask = (mask > 0.5).astype("float32")

    return img, mask
print("Executed")

Executed


In [5]:
def subset_generator(img_dir, mask_dir, files, batch_size):

    files = list(files)

    while True:
        np.random.shuffle(files)

        for i in range(0, len(files), batch_size):

            batch = files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch:

                img_path = os.path.join(img_dir, img_file)
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)
print("Done")

Done


In [6]:
def conv_block(x, filters):

    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x
print("Done")

Done


In [7]:
def attention_gate(x, g, filters):

    theta_x = layers.Conv2D(filters, 1, padding="same")(x)
    phi_g = layers.Conv2D(filters, 1, padding="same")(g)

    # Resize g to match x if needed
    if theta_x.shape[1] != phi_g.shape[1]:
        phi_g = layers.Resizing(
            theta_x.shape[1],
            theta_x.shape[2]
        )(phi_g)

    add = layers.Add()([theta_x, phi_g])
    add = layers.Activation("relu")(add)

    psi = layers.Conv2D(1, 1, padding="same")(add)
    psi = layers.Activation("sigmoid")(psi)

    return layers.Multiply()([x, psi])


In [8]:
def build_attention_unet(input_shape=(IMG_SIZE, IMG_SIZE, 3)):

    inputs = layers.Input(input_shape)

    encoder = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs
    )

    # Correct skip layers
    s1 = encoder.get_layer("block2a_activation").output   # 64x64
    s2 = encoder.get_layer("block3a_activation").output   # 32x32
    s3 = encoder.get_layer("block4a_activation").output   # 16x16

    bridge = encoder.get_layer("top_activation").output   # 8x8

    # Decoder

    # 8 → 16
    d1 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(bridge)
    s3 = attention_gate(s3, d1, 256)
    d1 = layers.Concatenate()([d1, s3])
    d1 = conv_block(d1, 256)

    # 16 → 32
    d2 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(d1)
    s2 = attention_gate(s2, d2, 128)
    d2 = layers.Concatenate()([d2, s2])
    d2 = conv_block(d2, 128)

    # 32 → 64
    d3 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(d2)
    s1 = attention_gate(s1, d3, 64)
    d3 = layers.Concatenate()([d3, s1])
    d3 = conv_block(d3, 64)

    # 64 → 128
    d4 = layers.Conv2DTranspose(32, 2, strides=2, padding="same")(d3)
    d4 = conv_block(d4, 32)

    # 128 → 256
    d5 = layers.Conv2DTranspose(16, 2, strides=2, padding="same")(d4)
    d5 = conv_block(d5, 16)

    outputs = layers.Conv2D(1, 1, activation="sigmoid")(d5)

    return models.Model(inputs, outputs)


In [9]:
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6

    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true_f * y_pred_f)

    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth
    )

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)


In [10]:
model = build_attention_unet()

# freeze pretrained encoder first
for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss=bce_dice_loss,
    metrics=["accuracy", dice_coefficient]
)

model.summary()

I0000 00:00:1771087270.214088  209434 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4132 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 256, 256,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 256, 256,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 257, 257,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 128, 128,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 128, 128,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 128, 128,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 128, 128,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │        512 │ block1a_se_excit

 Total params: 8,058,695 (30.74 MB)

 Trainable params: 8,014,688 (30.57 MB)

 Non-trainable params: 44,007 (171.91 KB)

In [11]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))


TensorFlow version: 2.20.0
GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
from sklearn.model_selection import train_test_split

# Split TRAIN into Train / Val

all_train_files = sorted([
    f for f in os.listdir(TRAIN_IMG)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

train_files, val_files = train_test_split(
    all_train_files,
    test_size=0.2,
    random_state=42
)

print("Train samples:", len(train_files))
print("Val samples:", len(val_files))


# Generators from subsets

def subset_generator(img_dir, mask_dir, files, batch_size):

    while True:
        np.random.shuffle(files)

        for i in range(0, len(files), batch_size):

            batch = files[i:i + batch_size]
            imgs, masks = [], []

            for img_file in batch:

                img_path = os.path.join(img_dir, img_file)
                mask_file = os.path.splitext(img_file)[0] + ".png"
                mask_path = os.path.join(mask_dir, mask_file)

                if not os.path.exists(mask_path):
                    continue

                img, mask = load_image_mask(img_path, mask_path)

                imgs.append(img)
                masks.append(mask)

            if len(imgs) > 0:
                yield np.array(imgs), np.array(masks)


train_gen = subset_generator(
    TRAIN_IMG,
    TRAIN_MASK,
    train_files,
    BATCH_SIZE
)

val_gen = subset_generator(
    TRAIN_IMG, 
    TRAIN_MASK,
    val_files,
    BATCH_SIZE
)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Stop if validation stops improving
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Save best model automatically
checkpoint = ModelCheckpoint(
    filepath="unetpp_best.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

Train samples: 3146
Val samples: 787


In [13]:
model.fit(
    train_gen,
    steps_per_epoch=len(train_files) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(val_files) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/25


2026-02-14 22:11:36.585323: I external/local_xla/xla/service/service.cc:163] XLA service 0x7ff980002390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-14 22:11:36.585343: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2026-02-14 22:11:37.263548: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-14 22:11:40.772326: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91801
2026-02-14 22:11:45.694878: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-14 22:11:45.782217: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel 

1573/1573 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9138 - dice_coefficient: 0.0428 - loss: 1.3408
Epoch 1: val_loss improved from None to 1.20385, saving model to unetpp_best.keras

Epoch 1: finished saving model to unetpp_best.keras
1573/1573 ━━━━━━━━━━━━━━━━━━━━ 99s 28ms/step - accuracy: 0.9519 - dice_coefficient: 0.0688 - loss: 1.2460 - val_accuracy: 0.9833 - val_dice_coefficient: 0.0269 - val_loss: 1.2038
Epoch 2/25
1571/1573 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9847 - dice_coefficient: 0.1481 - loss: 1.0345
Epoch 2: val_loss improved from 1.20385 to 1.09280, saving model to unetpp_best.keras

Epoch 2: finished saving model to unetpp_best.keras
1573/1573 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9861 - dice_coefficient: 0.1590 - loss: 1.0080 - val_accuracy: 0.9784 - val_dice_coefficient: 0.0967 - val_loss: 1.0928
Epoch 3/25
1572/1573 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9892 - dice_coefficient: 0.1926 - loss: 0.9427
Epoch 3: val_loss improved fro

2026-02-14 22:19:11.606693: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-14 22:19:11.698764: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-14 22:19:12.134419: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-14 22:19:12.224527: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.



Epoch 10: val_loss improved from 0.73086 to 0.68090, saving model to unetpp_best.keras

Epoch 10: finished saving model to unetpp_best.keras
1573/1573 ━━━━━━━━━━━━━━━━━━━━ 51s 32ms/step - accuracy: 0.9953 - dice_coefficient: 0.4422 - loss: 0.6009 - val_accuracy: 0.9914 - val_dice_coefficient: 0.3722 - val_loss: 0.6809
Epoch 11/25
1572/1573 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9959 - dice_coefficient: 0.4777 - loss: 0.5597
Epoch 11: val_loss did not improve from 0.68090
1573/1573 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9958 - dice_coefficient: 0.4847 - loss: 0.5520 - val_accuracy: 0.9842 - val_dice_coefficient: 0.0694 - val_loss: 1.0114
Epoch 12/25
1572/1573 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9961 - dice_coefficient: 0.5104 - loss: 0.5227
Epoch 12: val_loss improved from 0.68090 to 0.65790, saving model to unetpp_best.keras

Epoch 12: finished saving model to unetpp_best.keras
1573/1573 ━━━━━━━━━━━━━━━━━━━━ 46s 29ms/step - accuracy: 0.9960 - dice_coeffi

In [14]:
def predict_and_save(model, img_dir, mask_dir):

    os.makedirs(PRED_MASK_DIR, exist_ok=True)
    os.makedirs(OVERLAY_DIR, exist_ok=True)

    img_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    for file in img_files:

        # image path
        img_path = os.path.join(img_dir, file)
        
        # mask name
        mask_file = os.path.splitext(file)[0] + ".png"
        mask_path = os.path.join(mask_dir, mask_file)

        # skip if mask missing 
        if not os.path.exists(mask_path):
            print(f"Skipping (mask not found): {file}")
            continue

        # load image & mask 
        img, true_mask = load_image_mask(img_path, mask_path)

        # prediction 
        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_mask = (pred > 0.5).astype(np.uint8) * 255

        # save predicted mask 
        cv2.imwrite(
            os.path.join(PRED_MASK_DIR, mask_file),
            pred_mask.squeeze()
        )

        # overlay 
        img_vis = cv2.imread(img_path)
        img_vis = cv2.resize(img_vis, (IMG_SIZE, IMG_SIZE))

        overlay = img_vis.copy()
        overlay[pred_mask.squeeze() == 255] = [0, 0, 255]  # red tumor

        combined = cv2.addWeighted(img_vis, 0.7, overlay, 0.3, 0)

        cv2.imwrite(
            os.path.join(OVERLAY_DIR, file),
            combined
        )

    print("Predictions, masks & overlays saved successfully.")


In [15]:
predict_and_save(model, TEST_IMG, TEST_MASK)

Predictions, masks & overlays saved successfully.


In [16]:
model.save("brisc2025_unetpp_latest.keras")

In [17]:
from tensorflow.keras.models import load_model

model = load_model(
    "brisc2025_unetpp_latest.keras",
    custom_objects={
        "bce_dice_loss": bce_dice_loss,
        "dice_coefficient": dice_coefficient
    }
)

In [19]:
from tqdm import tqdm
import os
import numpy as np

# Metric helpers (NumPy)

def dice_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)

    return (2. * intersection + smooth) / (
        np.sum(y_true) + np.sum(y_pred) + smooth
    )


def iou_np(y_true, y_pred, smooth=1e-6):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection

    return (intersection + smooth) / (union + smooth)


def precision_recall_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)

    return precision, recall


def accuracy_np(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    correct = np.sum(y_true == y_pred)
    total = len(y_true)

    return correct / total


# Evaluate on test set

def evaluate_test_set(model, img_dir, mask_dir):

    image_files = sorted([
        f for f in os.listdir(img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    dice_scores = []
    iou_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    print(" Evaluating test set...")

    for file in tqdm(image_files):

        img_path = os.path.join(img_dir, file)

        mask_path = os.path.join(
            mask_dir,
            os.path.splitext(file)[0] + ".png"
        )

        if not os.path.exists(mask_path):
            continue

        img, gt_mask = load_image_mask(img_path, mask_path)

        pred = model.predict(img[np.newaxis, ...], verbose=0)[0]
        pred_bin = (pred > 0.5).astype(np.uint8)

        gt_bin = gt_mask.astype(np.uint8)

        dice_scores.append(dice_np(gt_bin, pred_bin))
        iou_scores.append(iou_np(gt_bin, pred_bin))

        p, r = precision_recall_np(gt_bin, pred_bin)
        precision_scores.append(p)
        recall_scores.append(r)

        accuracy_scores.append(
            accuracy_np(gt_bin, pred_bin)
        )

    print("\n===== TEST SET RESULTS =====")
    print(f"Mean Dice     : {np.mean(dice_scores):.4f}")
    print(f"Mean IoU      : {np.mean(iou_scores):.4f}")
    print(f"Mean Precision: {np.mean(precision_scores):.4f}")
    print(f"Mean Recall   : {np.mean(recall_scores):.4f}")
    print(f"Mean Accuracy : {np.mean(accuracy_scores):.4f}")

    return {
        "dice": dice_scores,
        "iou": iou_scores,
        "precision": precision_scores,
        "recall": recall_scores,
        "accuracy": accuracy_scores,
    }


# RUN EVALUATION
results = evaluate_test_set(
    model,
    TEST_IMG,
    TEST_MASK
)


 Evaluating test set...


100%|████████████████████████████████████████████████████████████████████████████████████████| 860/860 [00:48<00:00, 17.67it/s]


===== TEST SET RESULTS =====
Mean Dice     : 0.7652
Mean IoU      : 0.6803
Mean Precision: 0.7819
Mean Recall   : 0.7987
Mean Accuracy : 0.9936
